In [56]:
import os
import json
import gdown
import pandas as pd
import numpy as np
### Plot
import matplotlib.pyplot as plt
import seaborn as sns
### HTML
from bs4 import BeautifulSoup
import re
### Warnings
import warnings
### Text Preprocessing and Natural Language Processing
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
import nltk
import re
import spacy
from wordcloud import WordCloud

In [57]:
tmdb_df = pd.read_csv('TMDB_movie_dataset_v11.csv') #tmdb dataset we will be working with
tmdb_df
tmdb_df.describe()


,id,vote_average,vote_count,revenue,runtime,budget,popularity
count,1.002267e+06,1.002267e+06,1.002267e+06,1.002267e+06,1.002267e+06,1.002267e+06,1.002267e+06
mean,6.625906e+05,2.101624e+00,2.139664e+01,7.421149e+05,5.059751e+01,2.924065e+05,1.349021e+00
std,3.549163e+05,3.110503e+00,3.391106e+02,1.803934e+07,6.229370e+01,5.134357e+06,8.071102e+00
min,2.000000e+00,0.000000e+00,0.000000e+00,-1.200000e+01,-2.800000e+01,0.000000e+00,0.000000e+00
25%,3.705740e+05,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,6.000000e-01
50%,6.643460e+05,0.000000e+00,0.000000e+00,0.000000e+00,3.000000e+01,0.000000e+00,6.000000e-01
75%,9.721775e+05,5.000000e+00,1.000000e+00,0.000000e+00,9.000000e+01,0.000000e+00,9.140000e-01
max,1.254286e+06,1.000000e+01,3.449500e+04,3.000000e+09,1.440000e+04,9.000000e+08,2.994357e+03


In [58]:
# in order to create a robust key "title_year to join with the movielens dataset"
tmdb_df['release_date'] = pd.to_datetime(tmdb_df['release_date'], errors='coerce')
tmdb_df['year'] = tmdb_df['release_date'].dt.year.astype('Int64').astype(str).replace('nan', '')
tmdb_df['title_year'] = tmdb_df.apply(lambda x: f"{x['title']} ({x['year']})" if x['year'] else x['title'], axis=1)
tmdb_df


,id,title,vote_average,vote_count,status,release_date,revenue,runtime,adult,backdrop_path,...,overview,popularity,poster_path,tagline,genres,production_companies,production_countries,spoken_languages,year,title_year
0,27205,Inception,8.364,34495,Released,2010-07-15,825532764,148,False,/8ZTVqvKDQ8emSGUEMjsS4yHAwrp.jpg,...,"Cobb, a skilled thief who commits corporate es...",83.952,/oYuLEt3zVCKq57qu2F8dT7NIa6f.jpg,Your mind is the scene of the crime.,"Action, Science Fiction, Adventure","Legendary Pictures, Syncopy, Warner Bros. Pict...","United Kingdom, United States of America","English, French, Japanese, Swahili",2010,Inception (2010)
1,157336,Interstellar,8.417,32571,Released,2014-11-05,701729206,169,False,/pbrkL804c8yAv3zBZR4QPEafpAR.jpg,...,The adventures of a group of explorers who mak...,140.241,/gEU2QniE6E77NI6lCU6MxlNBvIx.jpg,Mankind was born on Earth. It was never meant ...,"Adventure, Drama, Science Fiction","Legendary Pictures, Syncopy, Lynda Obst Produc...","United Kingdom, United States of America",English,2014,Interstellar (2014)
2,155,The Dark Knight,8.512,30619,Released,2008-07-16,1004558444,152,False,/nMKdUUepR0i5zn0y1T4CsSB5chy.jpg,...,Batman raises the stakes in his war on crime. ...,130.643,/qJ2tW6WMUDux911r6m7haRef0WH.jpg,Welcome to a world without rules.,"Drama, Action, Crime, Thriller","DC Comics, Legendary Pictures, Syncopy, Isobel...","United Kingdom, United States of America","English, Mandarin",2008,The Dark Knight (2008)
3,19995,Avatar,7.573,29815,Released,2009-12-15,2923706026,162,False,/vL5LR6WdxWPjLPFRLe133jXWsh5.jpg,...,"In the 22nd century, a paraplegic Marine is di...",79.932,/kyeqWdyUXW608qlYkRqosgbbJyK.jpg,Enter the world of Pandora.,"Action, Adventure, Fantasy, Science Fiction","Dune Entertainment, Lightstorm Entertainment, ...","United States of America, United Kingdom","English, Spanish",2009,Avatar (2009)
4,24428,The Avengers,7.710,29166,Released,2012-04-25,1518815515,143,False,/9BBTo63ANSmhC4e6r62OJFuK2GL.jpg,...,When an unexpected enemy emerges and threatens...,98.082,/RYMX2wcKCBAr24UyPD7xwmjaTn.jpg,Some assembly required.,"Science Fiction, Action, Adventure",Marvel Studios,United States of America,"English, Hindi, Russian",2012,The Avengers (2012)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1002262,664753,Dead Kennedys: Live in Vienna,0.000,0,Released,1982-01-01,0,16,False,NaN,...,Dead Kennedys filmed live at Arena Wien in Vie...,0.600,/nQ9RmZhdYhv8QHCoHgNBfyqewB6.jpg,NaN,Music,Welder,United States of America,English,1982,Dead Kennedys: Live in Vienna (1982)
1002263,664754,Haunted House,0.000,0,Released,2019-05-31,0,53,False,NaN,...,Three Paranormal Investigators spend the night...,0.600,/q72Scu36kQzoZX8TQ9vxnCLqDk1.jpg,NaN,Documentary,NaN,United States of America,English,2019,Haunted House (2019)
1002264,664755,"Fuck You, Purdue",0.000,0,Released,1987-07-07,0,12,False,NaN,...,A 1987 video by Cecilia Dougherty,0.600,NaN,NaN,NaN,NaN,NaN,NaN,1987,"Fuck You, Purdue (1987)"
1002265,664756,These Are the Rules,0.000,0,Released,1983-03-03,0,5,False,NaN,...,"In These Are the Rules, Doug Hall enacted an a...",0.600,NaN,NaN,NaN,NaN,NaN,NaN,1983,These Are the Rules (1983)


In [59]:
df_movies = pd.read_csv('ml-1m/movies.csv', sep='::', engine='python', encoding='ISO-8859-1', header=None, names=['MovieID', 'Title', 'Genres']) #sohai give us 3 different files so need read them separately
print(df_movies.head())

   MovieID                               Title                        Genres
0        1                    Toy Story (1995)   Animation|Children's|Comedy
1        2                      Jumanji (1995)  Adventure|Children's|Fantasy
2        3             Grumpier Old Men (1995)                Comedy|Romance
3        4            Waiting to Exhale (1995)                  Comedy|Drama
4        5  Father of the Bride Part II (1995)                        Comedy


In [60]:
df_ratings = pd.read_csv('ml-1m/ratings.csv', sep='::', engine='python', encoding='ISO-8859-1', header=None, names=['UserID', 'MovieID', 'Rating', 'Timestamp']) # ^same
print(df_ratings.head())

   UserID  MovieID  Rating  Timestamp
0       1     1193       5  978300760
1       1      661       3  978302109
2       1      914       3  978301968
3       1     3408       4  978300275
4       1     2355       5  978824291


In [61]:
df_users = pd.read_csv('ml-1m/users.csv', sep='::', engine='python', encoding='ISO-8859-1', header=None, names=['UserID', 'Gender', 'Age', 'Occupation','ZipCode']) # ^same
print(df_users.head())

   UserID Gender  Age  Occupation ZipCode
0       1      F    1          10   48067
1       2      M   56          16   70072
2       3      M   25          15   55117
3       4      M   45           7   02460
4       5      M   25          20   55455


In [62]:
df_ml_combined = pd.merge(pd.merge(df_ratings, df_movies, on='MovieID'), df_users, on='UserID') #this is the merged version of the MovieLens Dataset
df_ml_combined

,UserID,MovieID,Rating,Timestamp,Title,Genres,Gender,Age,Occupation,ZipCode
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama,F,1,10,48067
1,1,661,3,978302109,James and the Giant Peach (1996),Animation|Children's|Musical,F,1,10,48067
2,1,914,3,978301968,My Fair Lady (1964),Musical|Romance,F,1,10,48067
3,1,3408,4,978300275,Erin Brockovich (2000),Drama,F,1,10,48067
4,1,2355,5,978824291,"Bug's Life, A (1998)",Animation|Children's|Comedy,F,1,10,48067
...,...,...,...,...,...,...,...,...,...,...
1000204,6040,1091,1,956716541,Weekend at Bernie's (1989),Comedy,M,25,6,11106
1000205,6040,1094,5,956704887,"Crying Game, The (1992)",Drama|Romance|War,M,25,6,11106
1000206,6040,562,5,956704746,Welcome to the Dollhouse (1995),Comedy|Drama,M,25,6,11106
1000207,6040,1096,4,956715648,Sophie's Choice (1982),Drama,M,25,6,11106


In [63]:
#now to finally merge the datasets
merged_df = pd.merge(df_ml_combined, tmdb_df, left_on='Title', right_on='title_year', how='left') #main dataset
merged_df
print(merged_df)   #use Genres from movielens

         UserID  MovieID  Rating  Timestamp  \
0             1     1193       5  978300760   
1             1      661       3  978302109   
2             1      914       3  978301968   
3             1     3408       4  978300275   
4             1     2355       5  978824291   
...         ...      ...     ...        ...   
1012483    6040     1091       1  956716541   
1012484    6040     1094       5  956704887   
1012485    6040      562       5  956704746   
1012486    6040     1096       4  956715648   
1012487    6040     1097       4  956715569   

                                          Title  \
0        One Flew Over the Cuckoo's Nest (1975)   
1              James and the Giant Peach (1996)   
2                           My Fair Lady (1964)   
3                        Erin Brockovich (2000)   
4                          Bug's Life, A (1998)   
...                                         ...   
1012483              Weekend at Bernie's (1989)   
1012484                 Cry

In [64]:
print(merged_df.columns)
# Counting how many NA values are in the 'UserID' column
user_id_na_count = merged_df['UserID'].isna().sum()

print("Number of NA values in the UserID column:", user_id_na_count)
# Assuming 'df' is your DataFrame
important_columns = ['Title', 'Genres', 'overview', 'tagline', 'original_language', 'release_date', 'runtime', 'vote_average', 'vote_count', 'production_companies', 'production_countries', 'spoken_languages', 'year']

# Check for any NA values in these columns
na_columns = merged_df[important_columns].isna().any()

# Print columns with NA values
print("Columns with NA values:", na_columns[na_columns].index.tolist())



Index(['UserID', 'MovieID', 'Rating', 'Timestamp', 'Title', 'Genres', 'Gender',
       'Age', 'Occupation', 'ZipCode', 'id', 'title', 'vote_average',
       'vote_count', 'status', 'release_date', 'revenue', 'runtime', 'adult',
       'backdrop_path', 'budget', 'homepage', 'imdb_id', 'original_language',
       'original_title', 'overview', 'popularity', 'poster_path', 'tagline',
       'genres', 'production_companies', 'production_countries',
       'spoken_languages', 'year', 'title_year'],
      dtype='object')
Number of NA values in the UserID column: 0
Columns with NA values: ['overview', 'tagline', 'original_language', 'release_date', 'runtime', 'vote_average', 'vote_count', 'production_companies', 'production_countries', 'spoken_languages', 'year']


In [65]:
# Assuming 'merged_df' is your DataFrame
content_merged_df = merged_df.drop(['genres', 'poster_path', 'backdrop_path','Age', 'Gender', 'homepage','Occupation', 'ZipCode'], axis=1)
content_merged_df = content_merged_df.drop_duplicates().reset_index(drop=True) #no duplicates since row numbers remained the same, timestamp used for temporal analysis
print(content_merged_df)
content_merged_df

         UserID  MovieID  Rating  Timestamp  \
0             1     1193       5  978300760   
1             1      661       3  978302109   
2             1      914       3  978301968   
3             1     3408       4  978300275   
4             1     2355       5  978824291   
...         ...      ...     ...        ...   
1012483    6040     1091       1  956716541   
1012484    6040     1094       5  956704887   
1012485    6040      562       5  956704746   
1012486    6040     1096       4  956715648   
1012487    6040     1097       4  956715569   

                                          Title  \
0        One Flew Over the Cuckoo's Nest (1975)   
1              James and the Giant Peach (1996)   
2                           My Fair Lady (1964)   
3                        Erin Brockovich (2000)   
4                          Bug's Life, A (1998)   
...                                         ...   
1012483              Weekend at Bernie's (1989)   
1012484                 Cry

,UserID,MovieID,Rating,Timestamp,Title,Genres,id,title,vote_average,vote_count,...,original_language,original_title,overview,popularity,tagline,production_companies,production_countries,spoken_languages,year,title_year
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama,510.0,One Flew Over the Cuckoo's Nest,8.420,9650.0,...,en,One Flew Over the Cuckoo's Nest,A petty criminal fakes insanity to serve his s...,36.673,"If he's crazy, what does that make you?",Fantasy Films,United States of America,English,1975,One Flew Over the Cuckoo's Nest (1975)
1,1,661,3,978302109,James and the Giant Peach (1996),Animation|Children's|Musical,10539.0,James and the Giant Peach,6.556,1196.0,...,en,James and the Giant Peach,When the young orphan boy James spills a magic...,27.497,Adventures this big don't grow on trees.,"Allied Filmmakers, Walt Disney Pictures",United States of America,English,1996,James and the Giant Peach (1996)
2,1,914,3,978301968,My Fair Lady (1964),Musical|Romance,11113.0,My Fair Lady,7.500,1161.0,...,en,My Fair Lady,A snobbish phonetics professor agrees to a wag...,19.547,The loverliest motion picture of them all!,Warner Bros. Pictures,United States of America,English,1964,My Fair Lady (1964)
3,1,3408,4,978300275,Erin Brockovich (2000),Drama,462.0,Erin Brockovich,7.417,2855.0,...,en,Erin Brockovich,A twice-divorced mother of three who sees an i...,22.495,She brought a small town to its feet and a hug...,Jersey Films,United States of America,English,2000,Erin Brockovich (2000)
4,1,2355,5,978824291,"Bug's Life, A (1998)",Animation|Children's|Comedy,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1012483,6040,1091,1,956716541,Weekend at Bernie's (1989),Comedy,8491.0,Weekend at Bernie's,6.373,918.0,...,en,Weekend at Bernie's,Two friends are invited for a weekend to a lux...,12.318,"Bernie may be dead, but he's still the life of...","Gladden Entertainment, 20th Century Fox",United States of America,English,1989,Weekend at Bernie's (1989)
1012484,6040,1094,5,956704887,"Crying Game, The (1992)",Drama|Romance|War,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1012485,6040,562,5,956704746,Welcome to the Dollhouse (1995),Comedy|Drama,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1012486,6040,1096,4,956715648,Sophie's Choice (1982),Drama,15764.0,Sophie's Choice,7.340,781.0,...,en,Sophie's Choice,"Stingo, a young writer, moves to Brooklyn in 1...",14.359,"Between the innocent, the romantic, the sensua...","ITC Entertainment, Keith Barish Productions, U...","United Kingdom, United States of America","Czech, German, French, Polish, Russian, English",1982,Sophie's Choice (1982)


In [66]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

# Combining "overview" and "tagline" into a single text feature for each movie
# Filling missing values with empty strings
content_merged_df['combined_text'] = content_merged_df['overview'].fillna('') + " " + content_merged_df['tagline'].fillna('')

# Initializing the TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words='english')

# Fitting and transforming the combined text data
tfidf_matrix = tfidf_vectorizer.fit_transform(content_merged_df['combined_text'])

from sklearn.preprocessing import OneHotEncoder

# Initializing the OneHotEncoder
onehot_encoder = OneHotEncoder()

# Selecting columns for one-hot encoding
columns_to_encode = ['original_language', 'Genres', 'production_companies', 'production_countries']

# Assuming each of these columns is a string of separated values (e.g., "English,Spanish"), you'll first need to split them
# For simplicity, this example assumes these fields are already properly formatted as single values or lists

# Applying one-hot encoding
encoded_features = pd.DataFrame()
for column in columns_to_encode:
    # The input to fit_transform needs to be reshaped if it's a single feature column
    reshaped = content_merged_df[column].fillna('Unknown').values.reshape(-1, 1)
    encoded = onehot_encoder.fit_transform(reshaped).toarray()
    
    # Creating a DataFrame with encoded features
    feature_names = [f"{column}_{cat}" for cat in onehot_encoder.categories_[0]]
    
    # Creating a DataFrame for the encoded features of this column
    encoded_df = pd.DataFrame(encoded, columns=feature_names)
    
    # Concatenating the new features to the encoded_features DataFrame
    encoded_features = pd.concat([encoded_features, encoded_df], axis=1)


KeyboardInterrupt: 

In [ ]:
#CONTENT BASED FILTERING
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Assuming merged_df is already loaded with your data

# Step 1: Preprocess the dataset with TF-IDF
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
merged_df['combined_features'] = merged_df[['Genres', 'spoken_languages', 'production_countries', 'production_companies', 'tagline', 'overview']].apply(lambda x: ' '.join(x.astype(str).values), axis=1)
tfidf_matrix = tfidf_vectorizer.fit_transform(merged_df['combined_features'])

# Function to create a user profile
def create_user_profile(user_id, merged_df, tfidf_matrix, rating_threshold=4):
    user_data = merged_df[merged_df['UserID'] == user_id]
    # Filter movies that the user has rated above the threshold
    high_rating_indices = user_data[user_data['Rating'] > rating_threshold].index.tolist()
    
    if not high_rating_indices:
        return np.array([])  # Return an empty array if no high-rated movies

    # We extract the rows from tfidf_matrix corresponding to high ratings
    # and compute the mean. Note: toarray() converts sparse matrix to dense
    # Ensure the result is a dense 2D array for compatibility with cosine_similarity
    user_profile = np.mean(tfidf_matrix[high_rating_indices].toarray(), axis=0)
    
    # Ensure user_profile is 2D: (1, number_of_features)
    user_profile = user_profile.reshape(1, -1)
    
    return user_profile


# Function to recommend movies based on the user profile
def recommend(user_profile, tfidf_matrix, merged_df, user_id, n_recommendations=5):
    if user_profile is None:
        return []

    # Calculate similarity between user profile and all movie profiles
    cosine_similarities = cosine_similarity(user_profile, tfidf_matrix)
    
    # Get indices sorted by similarity (descending)
    similar_indices = cosine_similarities.argsort().flatten()[-n_recommendations*2:]
    
    # Filter out movies the user has already watched/rated
    watched_movie_ids = set(merged_df[merged_df['UserID'] == user_id]['MovieID'])
    recommended_movie_ids = [idx for idx in similar_indices if merged_df.iloc[idx]['MovieID'] not in watched_movie_ids]
    
    # Limit to the top N recommendations
    recommended_movie_ids = recommended_movie_ids[-n_recommendations:]
    return merged_df.iloc[recommended_movie_ids]['MovieID'].values

# Example usage
user_id = 1  # Replace with an actual user ID
user_profile = create_user_profile(user_id, merged_df, tfidf_matrix)
recommended_movies = recommend(user_profile, tfidf_matrix, merged_df, user_id)
print("Recommended movies for user {}: {}".format(user_id, recommended_movies))


KeyboardInterrupt: 